In [1]:
# -*- coding: utf-8 -*-
# import
import warnings
import itertools
import random
import pandas as pd
import numpy as np
from pandas import DataFrame
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.tsa.stattools as ts
from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_squared_error
plt.style.use('fivethirtyeight')

#Define the function meboot
def mean(L):
    return sum(L)/len(L)


def meboot(L, J=1):
    """
    Returns list of J maximum entropy bootstrap samples of time-series L
    """
    N = len(L)
    L_sort = sorted((e,i) for i,e in enumerate(L))
    L_vals = [l[0] for l in L_sort]
    L_inds = [l[1] for l in L_sort]
    L_out = [0]*J
    for j in range(J):
        Z = [(L_vals[i] + L_vals[i+1])/2 for i in range(N-1)]
        m_trm = mean([abs(L[i] - L[i-1]) for i in range(1, N)])
        Z = [L_vals[0] - m_trm] + Z + [L_vals[-1] + m_trm]
        m = [0]*N
        m[0] = 0.75*L_vals[0] + 0.25*L_vals[1]
        for k in range(1, N-1):
            m[k] = 0.25*L_vals[k-1] + 0.5*L_vals[k] + 0.25*L_vals[k+1]
        m[-1] = 0.25*L_vals[-2] + 0.75*L_vals[-1]
        U = sorted([random.random() for _ in range(N)])
        quantiles = [0]*N
        x = [float(y)/N for y in range(N+1)]
        for k in range(N):
            ind = min(range(len(x)), key=lambda i: abs(x[i] - U[k]))
            if x[ind] > U[k]:
                ind -= 1
            c = (2*m[ind] - Z[ind] - Z[ind + 1]) / 2
            y0 = Z[ind] + c
            y1 = Z[ind + 1] + c
            quantiles[k] = y0 + (U[k] - x[ind]) * \
                            (y1 - y0) / (x[ind + 1] - x[ind])
        L_out[j] = [x for y, x in sorted(zip(L_inds, quantiles))]
    return L_out

/anaconda/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
filename = r'/Users/Andy/Desktop/Bitcoin/Data/Meboot_Arima.csv'
data_bitcoin = pd.read_csv(filename)
#data_12_1 = pd.read_csv(filename, index_col = u'Date')
#data_12_2 = pd.read_csv(filename, index_col = u'New Arrivals')
#data_12_NewArrivals = np.array(data_12_1)
#data_12_Date = np.array(data_12_2)

In [3]:
data_bitcoin

,date,close
0,4/1/11,0.2989
1,5/1/11,0.2990
2,6/1/11,0.2980
3,7/1/11,0.3200
4,8/1/11,0.3229
5,9/1/11,0.3230
6,10/1/11,0.3266
7,11/1/11,0.3266
8,12/1/11,0.3188
9,13/1/11,0.3176
